In [1]:
import numpy as np
import pandas as pd
from python_module.pricing_model import BSMModel, SABRModel
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import plotly.express as px

In [2]:
# Inputs
F = 100
T = 1
alpha = 0.2
beta = 1
rho = -0.5
nu = 0.9
r = 0

In [3]:
# Generate options
results = dict()
for K in range(60, 141, 1):
    option_type = 'call' if K > F else 'put'
    out = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type=option_type)
    out['option_type'] = option_type
    out['delta_call'] = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type='call')['delta']
    results[K] = out

# Compute COGWA
df = pd.DataFrame(results)
df = df.transpose()
X = df[['gamma', 'vanna', 'volga']]
y = df['theta']
model = LinearRegression(fit_intercept=False)
model.fit(X.values, y.values)
y_pred = model.predict(X.values)
coef = model.coef_
r2 = r2_score(y.values, y_pred)
cogwa = (X * (coef))
cogwa.columns = ['costof_gamma', 'costof_vanna', 'costof_volga']
df = pd.concat([df, cogwa], axis=1)

# Generate Weights
df['weight'] = (1 / np.power(df.index, 2)) * 1_000_000

# Delta Bucketing
bins = [0, 0.25, 0.5, 0.75, 1]
labels = ['0-0.25', '0.25-0.5', '0.5-0.75', '0.75-1']
df['delta_bucket'] = pd.cut(df['delta_call'], bins=bins, labels=labels)

# Compute risk adjusted to quantity
risk_ref = 'theta'
df[f'pos_{risk_ref}'] = df[risk_ref] * df['weight']

# Compute quantity reduction with discretionay approch
df['risk_bucket'] = list(df['delta_bucket'].map(df.groupby('delta_bucket', observed=True)[f'pos_{risk_ref}'].sum().to_dict()))
df['qty_to_trade'] = df['risk_bucket'] / df[risk_ref]
df['min_qty_to_trade'] = list(df['delta_bucket'].map(df.groupby('delta_bucket', observed=True)['qty_to_trade'].min().to_dict())) 
df['weight_reduction'] = (df['qty_to_trade']==df['min_qty_to_trade']) * df['min_qty_to_trade']
df['weight_reduction'] = df['weight_reduction'].replace(0, np.nan)

# Compute quantity reduction with lasso
X = df[['costof_gamma', 'costof_vanna', 'costof_volga']].transpose()
y = df[['costof_gamma', 'costof_vanna', 'costof_volga']].multiply(df['weight'], axis=0).sum()
max_iteration = 100
alpha = 0.001
for i in range(max_iteration):
    alpha = alpha + 0.001
    lasso = Lasso(alpha=alpha, max_iter=3000)
    lasso.fit(X.values, y)
    lasso.coef_
    if np.sum(lasso.coef_!=0) == 4:
        print('found alpha:', alpha)
        break
lasso = Lasso(alpha=alpha, max_iter=3000)
lasso.fit(X.values, y)
df['lasso'] = lasso.coef_
df['lasso'] = df['lasso'].replace(0, np.nan)

# Compute quantity reduction with matrix inversion approach
# Compute coefficients for X and y (align indices)
X_sub = df.loc[df['weight_reduction'].dropna().index, ['costof_gamma','costof_vanna','costof_volga']].transpose()
y_sub = y.loc[X_sub.index] 
y_sub = y.loc[X_sub.index]     # make sure y aligns with X
X_sub = X_sub.iloc[:, 1:]
X_sub = X_sub.astype(float)
y_sub = y_sub.astype(float)
weight = np.linalg.solve(X_sub, y_sub)
df.loc[X_sub.columns, 'weight_matrix_inv'] = weight

found alpha: 0.005


In [4]:
display(df[['costof_gamma', 'costof_vanna', 'costof_volga']].multiply(df['weight_reduction'], axis=0).sum())
display(df[['costof_gamma', 'costof_vanna', 'costof_volga']].multiply(df['lasso'], axis=0).sum())
display(df[['costof_gamma', 'costof_vanna', 'costof_volga']].multiply(df['weight'], axis=0).sum())
display(df[['costof_gamma', 'costof_vanna', 'costof_volga']].multiply(df['weight_matrix_inv'], axis=0).sum())

costof_gamma   -103.077888
costof_vanna     -1.481548
costof_volga     -8.117367
dtype: object

costof_gamma    -71.53363
costof_vanna    16.836256
costof_volga    -2.637804
dtype: object

costof_gamma   -89.574235
costof_vanna    -0.021209
costof_volga   -19.614014
dtype: object

costof_gamma   -89.574235
costof_vanna    -0.021209
costof_volga   -19.614014
dtype: object

In [5]:
px.scatter(df[['weight', 'weight_matrix_inv', 'weight_reduction', 'lasso']])